In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:

# Declare available methods in local_binary_pattern
# from https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.local_binary_pattern
METHODS = ['default', 'ror','uniform','nri_uniform','var']
METHOD_INDEX = 0

# Declare the path to data feature
DATA_LBP_PATH = 'dataset/fish_data_lbp/resize/'
DATA_HOG_PATH = 'dataset/fish_data_hog'
# DATA_HL_PATH = 'dataset/fish_data_houghline'



In [ ]:
# import the data feature into pandas

# uncomment line below if using lbp
df = pd.read_csv(os.path.join(DATA_LBP_PATH,METHODS[METHOD_INDEX],METHODS[METHOD_INDEX]+'_dense.csv'), header=None)

# uncomment line below if using hog
# df = pd.read_csv(os.path.join(DATA_HOG_PATH,'hog_fish.csv'), header=None)

# for some reason there are typo for the class gymnocranius_ microdon
# it needs to be removed so there are no duplicate data
df = df.drop(df[df[1]=='gymnocranius_ microdon'].index)
df.head()

In [ ]:
# set the index column for the class and print the length
COLUMN_CLASS = 1
len(df[COLUMN_CLASS].unique())

In [ ]:
# Processing One hot encoded for the class
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(df[COLUMN_CLASS].unique())
new_class = lb.transform(df[COLUMN_CLASS])
new_class.shape

In [ ]:
# set list of column of features
index_input =[i for i in range(4,len(df.columns))]

In [ ]:
# import the MLP and train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# split the data into test and train data
X_train, X_test, y_train, y_test = train_test_split(df[index_input].copy().to_numpy(), new_class, 
                                                    test_size=0.3,random_state=1,stratify=new_class)

In [ ]:
# Make MLP model and fit (train) it
clf = MLPClassifier(hidden_layer_sizes=(400,500,400,350,250),solver='adam',random_state=1,n_iter_no_change=100, max_iter=1000, learning_rate_init=0.001).fit(X_train, y_train)

In [ ]:
# show the loss training in line diagram
import matplotlib.pyplot as plt

plt.plot(clf.loss_curve_, 'r')
plt.show()

In [ ]:
# save the model
import pickle
filename = 'model/lbp_11.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
# test accuracy of the model using test data
clf.score(X_test, y_test)